In [2]:
import os 
os.getcwd()

'd:\\ML_MODEL\\MLOPS_PROJECT\\project_youtube_3\\COmplete_ML_Pipeline\\complete-ml-pipeline\\research'

In [3]:
os.chdir("..")

In [4]:
os.getcwd()

'd:\\ML_MODEL\\MLOPS_PROJECT\\project_youtube_3\\COmplete_ML_Pipeline\\complete-ml-pipeline'

In [16]:
import numpy as np 
import pandas as pd
import yaml


In [6]:
from sklearn.linear_model import LinearRegression,ElasticNet,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv('data/car_price_prediction_.csv')
df.head(1)


,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,Result
0,0.62561,-0.813848,-0.397234,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26613.92


In [8]:
x_df = df.iloc[:,:-1]
y_df = df.iloc[:,-1]

In [9]:
x_train,x_test,y_train,y_test = train_test_split(x_df,y_df,test_size=0.2)

In [10]:
def evaluate(y_actual,y_predict):
    matrix = {
        'rmse' : np.sqrt(mean_squared_error(y_true=y_actual,y_pred=y_predict)),
        'mae' : mean_absolute_error(y_actual,y_predict),
        'r2' : r2_score(y_actual,y_predict)
    }
    print(f"Metrics: {matrix['r2']}")
    return sum(matrix.values())

In [11]:
models = {
    'LR' : LinearRegression(),
    'Lasso' : Lasso(),
    'Ridge' : Ridge(),
    'ElasticNet' : ElasticNet(),
    'DecisionTree' : DecisionTreeRegressor(),
    'RandomForest' : RandomForestRegressor(),
    'SVR' : SVR(),
    'GradientBoosting' : GradientBoostingRegressor(),
    
}

In [18]:
params = {
    'LR':{
        'fit_intercept' : [True]

    },
    'Lasso' : {
        'alpha' :[1,3],
        'max_iter' : [1000,1200]
    },
    'Ridge' : {
        'alpha' : [1,5],
    },
    'ElasticNet' : {
        'alpha' : [1,3],
        'l1_ratio' : [0.5,0.7]
    },
    'DT' : {
        'max_depth' : [5,6],
        'criterion' : ['squared_error','absolute_error'],
        'max_leaf_nodes' : [10,20]
    },
    'RF' : {
        'n_estimators' : [10,20],
        'criterion' : ['squared_error','absolute_error'],
        'max_depth' : [4,5]
    },
    'SVM' : {
        'kernel' : ['poly', 'rbf', 'sigmoid'],
        'gamma' : ['scale', 'auto'],
        'verbose' : [True] 
    },
    'GBM' : {}
}

with open('model_param.yaml', 'w') as f:
   yaml.safe_dump(params,f)

In [13]:
# import mlflow

# mlflow.set_tracking_uri('https://dagshub.com/amankumarchy5423/complete-ml-pipeline.mlflow')
# mlflow.set_experiment('complete-ml-pipeline')

import dagshub
dagshub.init(repo_owner='amankumarchy5423', repo_name='complete-ml-pipeline', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)


Accessing as amankumarchy5423

Initialized MLflow to track repo "amankumarchy5423/complete-ml-pipeline"

Repository amankumarchy5423/complete-ml-pipeline initialized!

🏃 View run selective-ram-259 at: https://dagshub.com/amankumarchy5423/complete-ml-pipeline.mlflow/#/experiments/0/runs/b1bbd92951e546758e755a6571436c5f
🧪 View experiment at: https://dagshub.com/amankumarchy5423/complete-ml-pipeline.mlflow/#/experiments/0


In [ ]:
# mlflow.autolog()
mlflow.set_experiment("my_experiment")
for i in range(len(list(models))):
    model = list(models.values())[i]
    model_name = list(models.keys())[i]
    model_params = params.get(list(models.keys())[i], {})

    with mlflow.start_run(run_name=f"{model_name}_run") as parent:
        grid = GridSearchCV(model, model_params, cv=5)
        grid.fit(x_train, y_train)

        for i in range (len(grid.cv_results_['params'])):
            with mlflow.start_run(run_name=f"{model_name}_run") as child:
                mlflow.log_param(grid.cv_results_['params'][i])

                y_pred = grid.predict(x_test)
                accuracy = evaluate(y_test, y_pred)
                mlflow.log_metric("accuracy", accuracy)

                print(f"Model {model_name} Accuracy: {accuracy}\n")
                
    mlflow.sklearn.log_model(grid.best_estimator_, "model", registered_model_name="complete-ml-model")

        # mlflow.sklearn.log_model(grid.best_estimator_, artifact_path="best_model")

    

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.model_selection import GridSearchCV
from mlflow.tracking import MlflowClient

# mlflow.set_experiment("my_experimet")
client = MlflowClient()

best_score = float('-inf')
best_model = None
best_model_name = None
best_run_id = None



for i in range(len(models)):
    model = list(models.values())[i]
    model_name = list(models.keys())[i]
    model_params = params.get(model_name, {})

    with mlflow.start_run(run_name=f"{model_name}_parent_run") as parent_run:
        grid = GridSearchCV(model, model_params, cv=5,verbose=2)
        grid.fit(x_train, y_train)

        for j in range(len(grid.cv_results_['params'])):
            with mlflow.start_run(run_name=f"{model_name}_child_run", nested=True) as child_run:
                # Log hyperparameters
                for param_name, param_value in grid.cv_results_['params'][j].items():
                    mlflow.log_param(param_name, param_value)

                # Use best estimator to predict
                y_pred = grid.predict(x_test)
                accuracy = evaluate(y_test, y_pred)
                mlflow.log_metric("accuracy", accuracy)

                print(f"Model {model_name} Accuracy: {accuracy}\n")

                # Save if this is the best model so far
                if accuracy > best_score:
                    best_score = accuracy
                    best_model = grid.best_estimator_
                    best_model_name = model_name
                    best_run_id = child_run.info.run_id

# ✅ Register the best model
model_uri = f"runs:/{best_run_id}/model"
registered_model_name = "Best-ml-model"

# Log the best model (outside the loop)
mlflow.sklearn.log_model(best_model, "model", registered_model_name=registered_model_name)

# ✅ Automatically promote it to Production
latest_version = client.get_latest_versions(name=registered_model_name, stages=["None"])[-1].version

client.transition_model_version_stage(
    name=registered_model_name,
    version=latest_version,
    stage="Production",
    archive_existing_versions=True
)

print(f"✅ {best_model_name} with accuracy {best_score} promoted to Production!")


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .................................fit_intercept=True; total time=   0.0s
[CV] END .................................fit_intercept=True; total time=   0.0s
[CV] END .................................fit_intercept=True; total time=   0.0s
[CV] END .................................fit_intercept=True; total time=   0.0s
[CV] END .................................fit_intercept=True; total time=   0.0s
Metrics: -0.02318070589058352
Model LR Accuracy: 52062.55324465333

🏃 View run LR_child_run at: https://dagshub.com/amankumarchy5423/complete-ml-pipeline.mlflow/#/experiments/0/runs/7b952926e7ea45f791baf0d66dc1ba1a
🧪 View experiment at: https://dagshub.com/amankumarchy5423/complete-ml-pipeline.mlflow/#/experiments/0
🏃 View run LR_parent_run at: https://dagshub.com/amankumarchy5423/complete-ml-pipeline.mlflow/#/experiments/0/runs/07c14d296b7d4486bf88a91696af7874
🧪 View experiment at: https://dagshub.com/amankumarchy5423/complete-ml

2025/04/18 15:33:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best-ml-model'.
2025/04/18 15:33:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best-ml-model, version 1
Created version '1' of model 'Best-ml-model'.
C:\Users\HP\AppData\Local\Temp\ipykernel_12656\498570374.py:51: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_version = client.get_latest_versions(name=registered_model_name, stages=["None"])[-1].version
C:\Users\HP\AppData\Local\Temp\ipykernel_1

✅ DecisionTree with accuracy 73468.35036592433 promoted to Production!


In [15]:
model = mlflow.sklearn.load_model("models:/Best-ml-model/Production")


d:\ML_MODEL\MLOPS_PROJECT\project_youtube_3\COmplete_ML_Pipeline\complete-ml-pipeline\myvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
import mlflow
logged_model = 'runs:/40371064a26146f897fe345111ac81fc/best_model'
pre_model_uri= 'runs:/44392372c4fc4a4ea15cc8c4481f424d/pre_processor_model'

# Load model as a PyFuncModel.
# loaded_model = mlflow.sklearn.load_model(logged_model)
pre_model = mlflow.sklearn.load_model(pre_model_uri)
# y_pred1 = loaded_model.predict(x_test)
# accu = r2_score(y_test,y_pred1)
# print(accu)


# Predict on a Pandas DataFrame.


In [33]:
import joblib

In [34]:
pre_model = joblib.load('my_model/preprocessor.pkl')

In [35]:
mlflow.sklearn.log_model(pre_model,artifact_path='pre_processor_model')

2025/04/18 12:31:01 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/04/18 12:31:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [9]:
len(list(models))

8

In [19]:
data = pd.read_csv('Artifact/Data_Transformation/train.csv')
data.isnull().sum()

0          0
1          0
2          0
3          0
4          0
5          0
6          0
7          0
8          0
9          0
10         0
11         0
12         0
13         0
14         0
15         0
16         0
17         0
18         0
19         0
20         0
21         0
22         0
23         0
24         0
25         0
26         0
27         0
28         0
29         0
30         0
31         0
32         0
33         0
34         0
35         0
36         0
37         0
38         0
39         0
40         0
41         0
42         0
43         0
44         0
45         0
46         0
Price    399
dtype: int64